In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
from sklearn.model_selection import train_test_split

move_bins_for_WCL=np.arange(0,150,5)
eval_bins=np.arange(-20.05,20.15,0.1)
move_bins_for_eval=move_bins_for_WCL


### Get List of Games, Train Test Split

In [ ]:
import anal_games, functions_anal

# Get the list of filenames matching the patterns
filenames_15 = glob.glob("../Cleaned_Analyzed_Games/twic*_15_processed.csv")
filenames_16 = glob.glob("../Cleaned_Analyzed_Games/twic*_16_processed.csv")

# For dupes, use the bigger depth
filenames_to_process=filenames_16
for file in filenames_15:
    if '_'.join(file.split('_')[:3])+'_16_processed.csv' in filenames_to_process:
        continue
    else:
        filenames_to_process.append(file)

outfile='../Cleaned_Analyzed_Games/all_games_cleaned.csv'

# make list of games 
anal_games.process_all_files(outfile=outfile,filenames=filenames_to_process,functions=[functions_anal.MovesTotal,functions_anal.Cleanup,functions_anal.MovesBlack,functions_anal.MovesWhite],skip_if_processed=True,game_wise=True)

df=pd.read_csv(outfile)

df_train,df_test=train_test_split(df,test_size=0.2,random_state=100) # stratification with number of moves or elos doesn't work, as it needs at least two games for each unique value/combination of values. Binning doesn't help

# save training set 
df_train.to_csv('../Cleaned_Analyzed_Games/all_games_cleaned_train.csv',index=False)
df_test.to_csv('../Cleaned_Analyzed_Games/all_games_cleaned_test.csv',index=False)

### Get Winning, Losing and Drawing Chances from Training set

In [ ]:
import winning_chances_util

# make bins for evaluations
eval_bins=np.arange(-20.05,20.15,0.1)

# make bins for moves
move_bins_for_WCL=np.arange(0,150,5)

# Compute winning chance tables from training data
df=pd.read_csv('../Cleaned_Analyzed_Games/all_games_cleaned_train.csv')
winning_chances_util.compute_winning_chance_table(df, intervals=eval_bins,movebins=move_bins_for_WCL,outdir='../winning_chances_tables')

# Same, but only one move bin
winning_chances_util.compute_winning_chance_table(df, intervals=eval_bins,movebins='all',outdir='../winning_chances_tables')


### Calculate Winning Chance Loss for each Game

In [ ]:
import copy
import concurrent.futures
import winning_chances_util
import anal_games

df=pd.read_csv('../Cleaned_Analyzed_Games/all_games_cleaned_train.csv')

process_by_move=True # do we want to have one winning table, or one for each move bracket?

num_workers=15

# for win chances binned by move
move_bins_for_eval=move_bins_for_WCL
wc_tables=winning_chances_util.read_winning_tables(dir='../winning_chances_tables/',movebins=move_bins_for_WCL)
args = []
i_process=0

for i in range(len(move_bins_for_eval)-1):

    
    wc_tables_new=copy.deepcopy(wc_tables)

    # bin moves
    wc_tables_new['mv_min']=move_bins_for_eval[i]
    wc_tables_new['mv_max']=move_bins_for_eval[i+1]

    # only process games with moves in bin
    df_moves=df.where(df['MovesAll']>=move_bins_for_eval[i])
    df_moves=df_moves.where(df['MovesAll']<move_bins_for_eval[i+1])
    df_moves.dropna(how='any',inplace=True)

    if df_moves.shape[0]==0:
        continue

    args.append(('../Cleaned_Analyzed_Games/wcl_train_'+str(move_bins_for_eval[i])+'-'+str(move_bins_for_eval[i+1])+'.csv',df_moves,[(winning_chances_util.WinChanceIncrease,wc_tables_new)],False,True))
    i_process+=1

wc_tables_new=copy.deepcopy(wc_tables)
wc_tables_new['mv_min']=move_bins_for_eval[-1]
wc_tables_new['mv_max']=100000
args.append(('../Cleaned_Analyzed_Games/wcl_train_'+str(move_bins_for_eval[-1])+'-.csv',df_moves,[(winning_chances_util.WinChanceIncrease,wc_tables_new)],False,True))
i_process+=1

for i in range(len(move_bins_for_eval)-1):

    
    wc_tables_new=copy.deepcopy(wc_tables)

    # bin moves
    wc_tables_new['mv_min']=move_bins_for_eval[i]
    wc_tables_new['mv_max']=move_bins_for_eval[i+1]

    # only process games with moves in bin
    df_moves=df.where(df['MovesAll']>=move_bins_for_eval[i])
    df_moves=df_moves.where(df['MovesAll']<move_bins_for_eval[i+1])
    df_moves.dropna(how='any',inplace=True)

    if df_moves.shape[0]==0:
        continue

    args.append(('../Cleaned_Analyzed_Games/wcl_test_'+str(move_bins_for_eval[i])+'-'+str(move_bins_for_eval[i+1])+'.csv',df_moves,[(winning_chances_util.WinChanceIncrease,wc_tables_new)],False,True))
    i_process+=1

wc_tables_new=copy.deepcopy(wc_tables)
wc_tables_new['mv_min']=move_bins_for_eval[-1]
wc_tables_new['mv_max']=100000
args.append(('../Cleaned_Analyzed_Games/wcl_test_'+str(move_bins_for_eval[-1])+'-.csv',df_moves,[(winning_chances_util.WinChanceIncrease,wc_tables_new)],False,True))
i_process+=1

# run it in parallel
with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
    tasks = []
    for argument in args:
        tasks.append(executor.submit(anal_games.process_game_list, argument[0],argument[1],argument[2],argument[3],argument[4]))

# for winchances not binned by move

movebins='all'

wc_tables=winning_chances_util.read_winning_tables(dir='../winning_chances_tables/',movebins=movebins)
args = []
i_process=0

for i in range(len(move_bins_for_eval)-1):

    
    wc_tables_new=copy.deepcopy(wc_tables)

    # bin moves
    wc_tables_new['mv_min']=move_bins_for_eval[i]
    wc_tables_new['mv_max']=move_bins_for_eval[i+1]

    # only process games with moves in bin
    df_moves=df.where(df['MovesAll']>=move_bins_for_eval[i])
    df_moves=df_moves.where(df['MovesAll']<move_bins_for_eval[i+1])
    df_moves.dropna(how='any',inplace=True)

    if df_moves.shape[0]==0:
        continue

    args.append(('../Cleaned_Analyzed_Games/wcl_train_all_'+str(move_bins_for_eval[i])+'-'+str(move_bins_for_eval[i+1])+'.csv',df_moves,[(winning_chances_util.WinChanceIncrease,wc_tables_new)],True,True))
    i_process+=1

wc_tables_new=copy.deepcopy(wc_tables)
wc_tables_new['mv_min']=move_bins_for_eval[-1]
wc_tables_new['mv_max']=100000
args.append(('../Cleaned_Analyzed_Games/wcl_train_all_'+str(move_bins_for_eval[-1]+5)+'-.csv',df_moves,[(winning_chances_util.WinChanceIncrease,wc_tables_new)],False,True))
i_process+=1

for i in range(len(move_bins_for_eval)-1):

    
    wc_tables_new=copy.deepcopy(wc_tables)

    # bin moves
    wc_tables_new['mv_min']=move_bins_for_eval[i]
    wc_tables_new['mv_max']=move_bins_for_eval[i+1]

    # only process games with moves in bin
    df_moves=df.where(df['MovesAll']>=move_bins_for_eval[i])
    df_moves=df_moves.where(df['MovesAll']<move_bins_for_eval[i+1])
    df_moves.dropna(how='any',inplace=True)

    if df_moves.shape[0]==0:
        continue

    args.append(('../Cleaned_Analyzed_Games/wcl_test_all_'+str(move_bins_for_eval[i])+'-'+str(move_bins_for_eval[i+1])+'.csv',df_moves,[(winning_chances_util.WinChanceIncrease,wc_tables_new)],True,True))
    i_process+=1

wc_tables_new=copy.deepcopy(wc_tables)
wc_tables_new['mv_min']=move_bins_for_eval[-1]
wc_tables_new['mv_max']=100000
args.append(('../Cleaned_Analyzed_Games/wcl_test_all_'+str(move_bins_for_eval[-1]+5)+'-.csv',df_moves,[(winning_chances_util.WinChanceIncrease,wc_tables_new)],False,True))
i_process+=1

with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
    tasks = []
    for argument in args:
        tasks.append(executor.submit(anal_games.process_game_list, argument[0],argument[1],argument[2],argument[3],argument[4]))


### Reshape output tables to have tables by player

In [ ]:
from make_WCL_table import WCL_by_player

# for winchances calculated with binned moves
for i in move_bins_for_eval[-1]:

    WCL_by_player(move_bins_for_eval[i],move_bins_for_eval[i+1],all=False,train=True)
    
WCL_by_player(move_bins_for_eval[-1],None,all=False,train=True)

for i in move_bins_for_eval[-1]:

    WCL_by_player(move_bins_for_eval[i],move_bins_for_eval[i+1],all=False,train=False)
    
WCL_by_player(move_bins_for_eval[-1],None,all=False,train=False)


# for winchances calculated with all moves
for i in move_bins_for_eval[-1]:

    WCL_by_player(move_bins_for_eval[i],move_bins_for_eval[i+1],all=True,train=True)
    
WCL_by_player(move_bins_for_eval[-1],None,all=True,train=True)

for i in move_bins_for_eval[-1]:

    WCL_by_player(move_bins_for_eval[i],move_bins_for_eval[i+1],all=True,train=False)
    
WCL_by_player(move_bins_for_eval[-1],None,all=True,train=False)

### Bin mistakes

In [ ]:
from make_mistakes_table import Mistakes_by_player

# make mistake bins and mistake labels
mistake_bins = [5, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 100]
# start move to count mistakes (counting starts at 0)
move_start=3

# for winchances calculated with binned moves
for i in range(len(move_bins_for_eval[:-1])):

    Mistakes_by_player(move_bins_for_eval[i],move_bins_for_eval[i+1],all=False,train=True,mistake_bins=mistake_bins,move_start=move_start)
    
Mistakes_by_player(move_bins_for_eval[-1],None,all=False,train=True,mistake_bins=mistake_bins,move_start=move_start)

for i in range(len(move_bins_for_eval[:-1])):

    Mistakes_by_player(move_bins_for_eval[i],move_bins_for_eval[i+1],all=False,train=False,mistake_bins=mistake_bins,move_start=move_start)
    
Mistakes_by_player(move_bins_for_eval[-1],None,all=False,train=False,mistake_bins=mistake_bins,move_start=move_start)

# for winchances calculated with all moves
for i in range(len(move_bins_for_eval[:-1])):

    Mistakes_by_player(move_bins_for_eval[i],move_bins_for_eval[i+1],all=True,train=True,mistake_bins=mistake_bins,move_start=move_start)
    
Mistakes_by_player(move_bins_for_eval[-1],None,all=True,train=True,mistake_bins=mistake_bins,move_start=move_start)

for i in range(len(move_bins_for_eval[:-1])):

    Mistakes_by_player(move_bins_for_eval[i],move_bins_for_eval[i+1],all=True,train=False,mistake_bins=mistake_bins,move_start=move_start)
    
Mistakes_by_player(move_bins_for_eval[-1],None,all=True,train=False,mistake_bins=mistake_bins,move_start=move_start)

### Data Analysis

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error


all=False

def get_filename(mv_start,mv_end,train,all):
    filename='../Cleaned_Analyzed_Games/wcl_and_mistakes_'
    if train:
        filename+='train_'
    else:
        filename+='test_'
    if all:
        filename+='all_'
    if mv_end==None:
        filename+=str(mv_start)
    else:
        filename+=str(mv_start)+'-'+str(mv_end)
    filename+='_by_player.csv'
    return filename


for i in [12]: #range(len(move_bins_for_eval[:-1])):

    mv_start=move_bins_for_eval[i]
    mv_end=move_bins_for_eval[i+1]

    filename=get_filename(mv_start,mv_end,train=True,all=all)

    df=pd.read_csv(filename)

    features=[]
    for i in range(1,(mv_start//2)):
        features.append('WCL_'+str(i))
    
    df_white=df.where(df['Player']=='White') # check which training games are in that file
    df_white.dropna(how='any',inplace=True)

    lr_white=LinearRegression()
    
    lr_white.fit(df_white[features],df_white['Elo'].astype(float))

    filename_test=get_filename(mv_start,mv_end,train=False,all=all)

    df_test=pd.read_csv(filename_test)

    df_test_white=df.where(df_test['Player']=='White') # check which training games are in that file
    df_test_white.dropna(how='any',inplace=True)

    print('White Coeffs',lr_white.coef_)
    print('White Intercept',lr_white.intercept_)

    elo_pred_white=lr_white.predict(df_test_white[features])
    residuals_white=df_test_white['Elo']-elo_pred_white
    print('RMS White ', root_mean_squared_error(df_test_white['Elo'], elo_pred_white))

    player_games=  df_test_white[(df_test_white['Elo'] >= 1400) & (df_test_white['Elo'] <= 1500)]

    player_games = player_games.head(20)

    out=lr_white.predict(player_games[features])

    print(out)

    plt.figure()
    plt.scatter(elo_pred_white,residuals_white,label='White',s=.1)
    # plt.scatter(elo_pred_black,residuals_black,label='Black',s=.1)
    plt.legend()
    plt.xlim([1750,2500])
    plt.savefig('Residuals_vs_predicted_'+str(mv_start)+'_'+str(mv_end)+'.png')

    size_move_bins=5

    




In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
import pandas as pd

from sklearn.metrics import mean_squared_error

models = {
    'lr': LinearRegression(),
    'svr': SVR(),
    'knr': KNeighborsRegressor(n_neighbors=10),
    'rf': RandomForestRegressor(),
    'ab': AdaBoostRegressor(),
    'gb': GradientBoostingRegressor(),
    'xbg': XGBRegressor()
}

all=False

def get_filename(mv_start,mv_end,train,all):
    filename='../Cleaned_Analyzed_Games/wcl_and_mistakes_'
    if train:
        filename+='train_'
    else:
        filename+='test_'
    if all:
        filename+='all_'
    if mv_end==None:
        filename+=str(mv_start)
    else:
        filename+=str(mv_start)+'-'+str(mv_end)
    filename+='_by_player.csv'
    return filename


for i in [12]: #range(len(move_bins_for_eval[:-1])):

    mv_start=move_bins_for_eval[i]
    mv_end=move_bins_for_eval[i+1]

    filename=get_filename(mv_start,mv_end,train=True,all=all)

    df=pd.read_csv(filename)

    features=[]
    for i in range(1,(mv_start//2)):
        features.append('WCL_'+str(i))
    target='Elo'
    
    df_white=df.where(df['Player']=='White') # check which training games are in that file
    df_white.dropna(how='any',inplace=True)

    filename_test=get_filename(mv_start,mv_end,train=False,all=all)

    df_test=pd.read_csv(filename_test)

    df_test_white=df.where(df_test['Player']=='White') # check which training games are in that file
    df_test_white.dropna(how='any',inplace=True)

    mses = {}
    for name, model in models.items():
        model.fit(df_white[features],df_white[target])
        mses[name] = mean_squared_error(df_test_white[target], model.predict(df_test_white[features]))

        player_games=  df_test_white[(df_test_white[target] >= 1400) & (df_test_white[target] <= 1500)]

        player_games = player_games.head(20)

        out=model.predict(player_games[features])

        print(name)

        print(out)

mses

lr
[2268.11798967 2167.61384801 2172.24597165 2155.42403829 2232.12909506
 2204.84716935 2333.79412945 2211.238637   2282.07036557 1804.9246347
 2287.29170785 2237.74430132 2285.50147578 2228.71933708 2145.62424253
 2249.21471697 2200.91591963 2155.40186094 2285.69892923 2259.87574242]
svr
[2283.25531481 2219.39499903 2224.53085701 2265.44032688 2255.69442236
 2247.03757337 2328.09539918 2273.5913567  2292.7696265  2241.81807299
 2269.99067321 2267.67289322 2277.40885216 2270.25960997 2226.22707886
 2266.74431316 2240.21994016 2205.67656859 2277.24523468 2280.72756957]
knr
[2171.4 2213.3 2047.5 1921.7 2202.4 2097.7 2210.5 2025.8 2245.7 2121.9
 2218.3 2095.7 2205.9 2122.2 2094.5 2028.1 1863.7 2149.8 2277.9 2164.4]
rf
[1746.13 1720.99 1735.54 1696.19 1651.68 1783.2  1829.9  1772.22 1815.43
 1641.37 1738.33 1748.63 1776.02 1695.83 1650.47 1751.28 1767.35 1728.14
 1800.29 1751.6 ]
ab
[2157.86634265 2076.7936898  2076.7936898  2000.19149909 2130.60107621
 2170.25473794 2214.48110831 2064.67

{'lr': 74853.00226464237,
 'svr': 77618.73119097012,
 'knr': 67056.46594336766,
 'rf': 10520.012804451848,
 'ab': 85518.98150609506,
 'gb': 69366.78211917859,
 'xbg': 37752.86746873225}